In [1]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("data/recall/top30/data/train_all_data.parquet")

save_dir = "model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/"
n_top =30

print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 203 欄位: ['total_num_transfers_rank', 'legs0_segments0_flightNumber', 'price_per_duration_rank', 'price_percentile', 'price_from_median_zscore', 'price_per_duration', 'legs0_segments0_baggageAllowance_quantity', 'price_minus_fee_rank', 'legs0_arrivalAt_hour', 'pricingInfo_isAccessTP', 'legs0_departureAt_hour', 'both_legs_carrier_all_same', 'total_weighted_mean_cabin', 'price_per_fee_rank', 'totalPrice_rank', 'days_before_departure', 'legs0_segments0_seatsAvailable', 'price_per_fee', 'price_per_tax', 'legs1_main_carrier', 'isVip', 'leg0_view_diff_mean', 'legs1_departureAt_hour', 'baggage_total', 'duration_ratio', 'total_fees', 'all_view_diff_mean', 'legs1_arrivalAt_hour', 'log_price', 'legs1_segments0_flightNumber', 'legs1_weighted_mean_cabin', 'companyID_loo_mean_legs0_departureAt_hour', 'legs0_segments0_cabinClass', 'price_minus_fee', 'legs0_main_carrier', 'companyID_loo_mean_legs0_arrivalAt_hour', 'baggage_total_rank', 'legs0_max_duration_cabin', 'legs0_segments0_key_

In [2]:
exclude_cols = [
    'Id', 'ranker_id', 'selected'
]

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')



Using 200 features


In [3]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]

# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])
val_rankers = set(unique_rankers[n_train:])

# 用 Polars 過濾 train/val
is_train = groups.select(pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train"))

# 先 concat mask
X_with_mask = X.with_columns(is_train)
y_with_mask = y.with_columns(is_train)
groups_with_mask = groups.with_columns(is_train)

# 分割 DataFrame
X_train_df = X_with_mask.filter(pl.col("is_train"))
X_val_df = X_with_mask.filter(~pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
y_val_df = y_with_mask.filter(~pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))
groups_val_df = groups_with_mask.filter(~pl.col("is_train"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_train").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_train").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_train").to_numpy().flatten()
# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)



In [5]:
import lightgbm as lgb
import os

# 📌 參數
lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [3],
    "max_depth": 13,
    "min_data_in_leaf": 1,      # ≈ min_child_weight
    "subsample": 0.9,
    "colsample_bytree": 1,
    "lambda_l2": 10.0,           # XGB 的 lambda
    "learning_rate": 0.05,
    "seed": 42,
    "num_threads": -1,
    "verbosity": 20
}

# 📌 建立 Dataset
dtrain = lgb.Dataset(
    X_train_np,
    label=y_train_np,
    feature_name = feature_cols,
    group=group_sizes_train
)
dval = lgb.Dataset(
    X_val_np,
    label=y_val_np,
    feature_name = feature_cols,
    group=group_sizes_val,
    reference=dtrain
)

# 📌 訓練
lgb_model = lgb.train(
    lgb_params,
    dtrain,
    num_boost_round=500,
    valid_sets=[dtrain, dval],
    valid_names=["train", "val"],
)

# 📌 儲存模型
model_path = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_path, exist_ok=True)
model_file = os.path.join(model_path, "lgb_ranker_model.txt")

lgb_model.save_model(model_file)
print(f"✅ 已儲存模型到 'lgb_ranker_model.txt' in {model_file}")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=13) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=8192) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=13) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=8192) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.910245
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.454120
[LightGBM] [Debug] init for col-wise cost 0.337108 seconds, init for row-wise cost 0.946599 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.671490 seconds.
You can set `force_row_wise=tru

In [6]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = lgb_model.predict(X_val_np)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)
import pandas as pd
import os

# 三種重要性對應 LightGBM
importance_types = {
    "weight": "split",    # 等於 XGBoost 的 "weight"
    "gain": "gain",       # 等於 XGBoost 的 "gain"
    # LightGBM 沒有 "cover"，只有 split/gain
}

importance_all = {}

for k, lgb_type in importance_types.items():
    imp_raw = lgb_model.feature_importance(importance_type=lgb_type)
    imp_named = dict(zip(lgb_model.feature_name(), imp_raw))
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[k] = sorted_imp

# weight榜
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

# gain榜
df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
)

# 補空值
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)

# 最小rank
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# Top50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank"]])

# 輸出CSV
csv_path = os.path.join(save_dir, f"top{n_top}/lgb_feature_importance.csv")
df_merged_sorted.to_csv(csv_path, index=False)
print(f"✅ 已輸出 feature_importance.csv 到 {csv_path}")


✅ HitRate@3 (groups size in [10, inf]): 0.5289
                                       feature  weight_rank      gain_rank
196                   total_num_transfers_rank          109  490070.549940
184                           price_percentile          619   90174.186762
48                      legs0_departureAt_hour          470   21651.390079
179                         price_per_duration          424   52083.784775
45                        legs0_arrivalAt_hour          444   20101.887677
190                            totalPrice_rank          297   48091.389245
0                        all_flight_view_count          391   44816.884673
36                      leg0_flight_view_count          190   47574.169966
181                              price_per_fee          371   46358.369783
176                   price_from_median_zscore          366   34889.333094
50                              legs0_duration          323   46309.838123
3                           all_view_diff_mean       

In [17]:
import xgboost as xgb
import gc

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
    feature_names = feature_cols,
)
dtrain.set_group(group_sizes_train)
del X_train_np, y_train_np, group_sizes_train
gc.collect()
dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
    feature_names = feature_cols,
)
dval.set_group(group_sizes_val)
del X_val_np, group_sizes_val
gc.collect()


7

In [18]:
import json
import os
# 參數
model_dir = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_dir, exist_ok=True)


xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 13,
    'min_child_weight': 1,
    'subsample': 0.9,
    'colsample_bytree': 1,
    'lambda': 1.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 儲存成JSON
params_path = os.path.join(model_dir, "xgb_params.json")

with open(params_path, "w") as f:
    json.dump(xgb_params, f, indent=4)

print(f"✅ 參數已存檔: {params_path}")


# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)
# 存成二進位模型
model_path = os.path.join(save_dir, f"top{n_top}/xgb_ranker_model.bin")

xgb_model.save_model(model_path)
print(f"✅ 已儲存模型到 'xgb_ranker_model.bin' in {model_path}")



✅ 參數已存檔: model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/xgb_params.json
[0]	train-ndcg@3:0.43698	val-ndcg@3:0.40613
[20]	train-ndcg@3:0.63138	val-ndcg@3:0.49220
[40]	train-ndcg@3:0.66993	val-ndcg@3:0.50265
[60]	train-ndcg@3:0.69771	val-ndcg@3:0.51185
[80]	train-ndcg@3:0.71482	val-ndcg@3:0.51805
[100]	train-ndcg@3:0.73651	val-ndcg@3:0.52548
[120]	train-ndcg@3:0.75681	val-ndcg@3:0.53226
[140]	train-ndcg@3:0.77333	val-ndcg@3:0.53845
[160]	train-ndcg@3:0.78591	val-ndcg@3:0.54018
[180]	train-ndcg@3:0.79754	val-ndcg@3:0.54364
[200]	train-ndcg@3:0.80784	val-ndcg@3:0.54657
[220]	train-ndcg@3:0.81560	val-ndcg@3:0.54697
[240]	train-ndcg@3:0.82346	val-ndcg@3:0.54952
[260]	train-ndcg@3:0.83400	val-ndcg@3:0.55215
[280]	train-ndcg@3:0.84792	val-ndcg@3:0.55495
[300]	train-ndcg@3:0.86569	val-ndcg@3:0.55777
[320]	train-ndcg@3:0.87969	val-ndcg@3:0.56012
[340]	train-ndcg@3:0.88666	val-ndcg@3:0.56279
[360]	train-ndcg@3:0.89130	val-ndcg@3:0.56381
[380]	train-ndcg@3:0.89421	v

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_30487/239021306.py:42: UserWarning: [18:40:56] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_path)


In [19]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = xgb_model.predict(dval)


# 計算 HitRate
hitrate = compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)
hitrate_records = []
hitrate_records.append({"split_label": "overall", "hitrate": hitrate})


hitrate_df = pl.DataFrame(hitrate_records)
csv_path = os.path.join(model_dir, "hitrate_summary.csv")
hitrate_df.write_csv(csv_path)
print(f"\n✅ 已儲存所有 Hitrate 結果至 {csv_path}")



✅ HitRate@3 (groups size in [10, inf]): 0.6032

✅ 已儲存所有 Hitrate 結果至 model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/hitrate_summary.csv


In [20]:
import pandas as pd

# 先準備特徵名稱
feature_names = feature_cols

# 取三種重要性
importance_types = ["weight", "gain", "cover"]
importance_all = {}

for imp_type in importance_types:
    imp_raw = xgb_model.get_score(importance_type=imp_type)
    imp_named = {}
    for k, v in imp_raw.items():
        imp_named[k] = v
    # 排序
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[imp_type] = sorted_imp

# 把三個榜單放成DataFrame方便比對
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover_rank"])
df_cover["cover_rank_pos"] = df_cover.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
    .merge(df_cover, on="feature", how="outer")
)

# 把不存在的rank補大數字
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

# 計算「三個榜單中最早出現的位置」
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# 取前50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank", "cover_rank"]])
# 如果想輸出CSV
csv_path = os.path.join(model_dir, "feature_importance.csv")

df_merged_sorted.to_csv(csv_path, index=False)
print(f"✅ 已輸出{csv_path}")


                                               feature  weight_rank  \
177                                   price_percentile      32395.0   
189                           total_num_transfers_rank        418.0   
53                                    legs0_mean_cabin         62.0   
169                           price_from_median_zscore      29496.0   
59   legs0_segments0_baggageAllowance_weightMeasure...          5.0   
32                                    is_popular_route         18.0   
64                        legs0_segments0_flightNumber      28523.0   
166                             miniRules1_statusInfos        453.0   
54                            legs0_num_transfers_rank        203.0   
172                                 price_per_duration      26600.0   
164                          miniRules1_monetaryAmount       1387.0   
178                             pricingInfo_isAccessTP       1699.0   
173                            price_per_duration_rank      26399.0   
28    

# Prediction

In [8]:
import os
import numpy as np
import polars as pl
import lightgbm as lgb

# 參數
model_path = "model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/lgb_ranker_model.txt"
parquet_path = "data/recall/top30/data/test_all_data.parquet"

# 讀取 LightGBM 模型
lgb_model = lgb.Booster(model_file=model_path)

# 取得 feature_names
model_features = lgb_model.feature_name()
if model_features is None or len(model_features) == 0:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選&排序
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測
preds = lgb_model.predict(X_np)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)

# 查看前幾筆
print(df_result.head())


[LightGBM] [Debug] Skipped header "tree" in file model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/lgb_ranker_model.txt
[LightGBM] [Debug] Skipped header "tree" in file model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/lgb_ranker_model.txt
[LightGBM] [Debug] Time for loading model: 0.019948 seconds
✅ 模型共 200 個features
✅ 讀取 test_filled，共 1019273 rows
✅ 預測完成，共 1019273 筆
shape: (5, 203)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ total_num ┆ legs0_seg ┆ price_per ┆ price_per ┆ … ┆ ranker_id ┆ companyID ┆ Id       ┆ selected  │
│ _transfer ┆ ments0_fl ┆ _duration ┆ centile   ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---       │
│ s_rank    ┆ ightNumbe ┆ _rank     ┆ ---       ┆   ┆ str       ┆ i64       ┆ i64      ┆ f64       │
│ ---       ┆ r         ┆ ---       ┆ f64       ┆   ┆           ┆           ┆          ┆           │
│ u32       ┆ ---       ┆ u32       ┆           ┆ 

In [21]:
import os
import numpy as np
import polars as pl
import xgboost as xgb

# 參數
model_path = "model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/xgb_ranker_model.bin"
parquet_path = "data/recall/top30/data/test_all_data.parquet"

# 讀取模型
xgb_model = xgb.Booster(model_file=model_path)

# 確認 feature_names
model_features = xgb_model.feature_names
if model_features is None:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選&排序
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測
dtest = xgb.DMatrix(X_np, feature_names=model_features)
preds = xgb_model.predict(dtest)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)

# 查看前幾筆
print(df_result.head())


✅ 模型共 200 個features
✅ 讀取 test_filled，共 1019273 rows
✅ 預測完成，共 1019273 筆
shape: (5, 203)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ total_num ┆ legs0_seg ┆ price_per ┆ price_per ┆ … ┆ ranker_id ┆ companyID ┆ Id       ┆ selected  │
│ _transfer ┆ ments0_fl ┆ _duration ┆ centile   ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---       │
│ s_rank    ┆ ightNumbe ┆ _rank     ┆ ---       ┆   ┆ str       ┆ i64       ┆ i64      ┆ f32       │
│ ---       ┆ r         ┆ ---       ┆ f64       ┆   ┆           ┆           ┆          ┆           │
│ u32       ┆ ---       ┆ u32       ┆           ┆   ┆           ┆           ┆          ┆           │
│           ┆ i32       ┆           ┆           ┆   ┆           ┆           ┆          ┆           │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 1         ┆ 456       ┆ 31        ┆ 0.212379  ┆ … ┆ c9373e5f7 ┆ 62840     ┆ 18144680 ┆ -0.881876 │
│   

In [9]:
import polars as pl

# 讀取完整 test parquet
original_test_path = "data/test.parquet"
df_id_rid = pl.read_parquet(original_test_path)

# 只要 Id 和 ranker_id
df_id_rid = df_id_rid.select(["Id", "ranker_id"])

print(f"✅ 原始 test data共有 {df_id_rid.height} 筆")

# 把預測結果中的 Id + prediction
df_preds = df_result.select(["Id", "selected"])

# Left join，把預測結果對應到所有Id
df_merged = (
    df_id_rid
    .join(
        df_preds,
        on="Id",
        how="left"
    )
    .with_columns(
        # 沒有預測結果的填-999
        pl.col("selected").fill_null(-999)
    )
)

print("✅ 完成補齊預測結果")
print(df_merged.head())

# 這樣就得到跟原始test一樣的行數，只有預測到的有分數

# 驗證
print("✅ 最後要輸出的資料:")
print(df_merged.head())

✅ 原始 test data共有 6897776 筆
✅ 完成補齊預測結果
shape: (5, 3)
┌──────────┬─────────────────────────────────┬───────────┐
│ Id       ┆ ranker_id                       ┆ selected  │
│ ---      ┆ ---                             ┆ ---       │
│ i64      ┆ str                             ┆ f64       │
╞══════════╪═════════════════════════════════╪═══════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.238999 │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    │
│ 18144683 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    │
└──────────┴─────────────────────────────────┴───────────┘
✅ 最後要輸出的資料:
shape: (5, 3)
┌──────────┬─────────────────────────────────┬───────────┐
│ Id       ┆ ranker_id                       ┆ selected  │
│ ---      ┆ ---                             ┆ ---       │
│ i64      ┆ str                             ┆ f64       │
╞══════════╪═════════════════════════

In [10]:
import os
import polars as pl

def export_submission_parquets(
    test_filled_with_preds: pl.DataFrame,
    output_dir: str,
    raw_filename: str = "raw_submission.parquet",
    ranked_filename: str = "rank_submission.parquet"
):
    """
    根據 test_filled_with_preds 輸出兩個 parquet:
    1. 原始分數 (selected)
    2. rank 排序 (selected)
    """
    # 檢查目錄
    os.makedirs(output_dir, exist_ok=True)

    # Subset + __index_level_0__
    subset_df = (
        test_filled_with_preds
        .select(["Id", "ranker_id", "selected"])
        .with_columns(
            pl.col("Id").alias("__index_level_0__")
        )
        .with_columns([
            pl.col("Id").cast(pl.Int64),
            pl.col("ranker_id").cast(pl.Utf8),
            pl.col("selected").cast(pl.Float64),
            pl.col("__index_level_0__").cast(pl.Int64)
        ])
    )

    # 儲存原始 parquet
    raw_path = os.path.join(output_dir, raw_filename)
    subset_df.write_parquet(raw_path)
    print(f"✅ 已儲存原始 submission: {raw_path}")
    print(subset_df)

    # Rank 排名
    ranked_df = subset_df.with_columns(
        pl.col("selected")
          .rank(method="ordinal", descending=True)
          .over("ranker_id")
          .alias("selected")
    )

    # 儲存排名 parquet
    ranked_path = os.path.join(output_dir, ranked_filename)
    ranked_df.write_parquet(ranked_path)
    print(f"✅ 已儲存rank submission: {ranked_path}")
    print(ranked_df)
    
n_top = 30
export_submission_parquets(
    test_filled_with_preds=df_merged,   # 你的帶有 selected 分數的 DataFrame
    output_dir=f"model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top{n_top}",
    ranked_filename = "rank_submission.parquet",
    raw_filename ="raw_submission.parquet",
)


✅ 已儲存原始 submission: model_output/selected_features_xgb/one_model_rerank/features_v1_with_all/top30/raw_submission.parquet
shape: (6_897_776, 4)
┌──────────┬─────────────────────────────────┬───────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected  ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---       ┆ ---               │
│ i64      ┆ str                             ┆ f64       ┆ i64               │
╞══════════╪═════════════════════════════════╪═══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.238999 ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    ┆ 18144682          │
│ 18144683 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -999.0    ┆ 18144683          │
│ …        ┆ …                               ┆ …         ┆ …      

In [3]:
from scripts.result_esemble import compute_ranked_average_selected_from_submissions

submission_paths = [
        "model_output/all_features_xgb/v1/v1_model/raw_submission.parquet",
        # "model_output/all_features_xgb/v1/v2_model/raw_submission.parquet", #0.486
        "model_output/all_features_xgb/v1/v2_1/raw_submission.parquet", #0.492
        "model_output/all_features_xgb/v1/v2_2/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v1/top60/raw_submission.parquet", # 0.49
        "model_output/selected_features_xgb/one_model/features_v1/top80/raw_submission.parquet",#0.49
        "model_output/selected_features_xgb/one_model/features_v1/top100/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v1/top120/raw_submission.parquet",
        
        "model_output/all_features_xgb/v2/v1_model/raw_submission.parquet", #0.4912
        'model_output/selected_features_xgb/one_model/features_v2/top120/raw_submission.parquet', # 0.4915
        # "model_output/selected_features_xgb/one_model/features_v2/top100/raw_submission.parquet", #0.486
        "model_output/selected_features_xgb/one_model/features_v2/top80/raw_submission.parquet",
        # "model_output/selected_features_xgb/one_model/features_v2/top60/raw_submission.parquet", 0.476
        
        # "model_output/all_features_xgb/v3/v1_model/raw_submission.parquet",
        # "model_output/selected_features_xgb/one_model/features_v3/top100/raw_submission.parquet", #0.479
        # "model_output/selected_features_xgb/one_model/features_v3/top80/raw_submission.parquet", #0.484
        
        "model_output/selected_features_xgb/one_model/features_v1_with_view/model_par_2/top100/raw_submission.parquet",# 0.49343 
        "model_output/selected_features_xgb/one_model/features_v1_with_view/top100/raw_submission.parquet",# 0.49242
        "model_output/selected_features_xgb/one_model/features_v1_with_view/top120/raw_submission.parquet", # 0.49343 
        "model_output/all_features_xgb/v1_add_flight_view/v1_model/raw_submission.parquet", #0.49141
        
        "model_output/all_features_xgb/v1_add_all/v1_model/raw_submission.parquet", #0.494
        "model_output/selected_features_xgb/one_model/features_v1_with_all/model_par_2/top120/raw_submission.parquet", #0.503
        "model_output/selected_features_xgb/one_model/features_v1_with_all/model_par_2/top100/raw_submission.parquet", # 0.4983
        "model_output/selected_features_xgb/one_model/features_v1_with_all/model_par_2/top80/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v1_with_all/model_par_2/top60/raw_submission.parquet",

        "model_output/selected_features_xgb/one_model/features_v1_with_all_umap/model_par_2/top100/raw_submission.parquet", #0.497
        "model_output/all_features_xgb/v1_add_all_umap/v1_model/raw_submission.parquet"
]

avg_df = compute_ranked_average_selected_from_submissions(submission_paths)

# 顯示前5筆
print(avg_df.head())

# 如需輸出
avg_df.write_parquet("submission_selected_mean.parquet")
print("✅ 已輸出平均submission")


✅ 讀取: model_output/all_features_xgb/v1/v1_model/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v1/v2_1/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v1/v2_2/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top60/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top80/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top100/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top120/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v2/v1_model/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v2/top120/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v2/top80/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1_with_view/model_par_2/top100/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/fea

In [ ]:
avg_df

Id,ranker_id,selected,__index_level_0__
i64,str,u32,i64
18144679,"""c9373e5f772e43d593dd6ad2fa90f6…",104,18144679
18144680,"""c9373e5f772e43d593dd6ad2fa90f6…",25,18144680
18144681,"""c9373e5f772e43d593dd6ad2fa90f6…",221,18144681
18144682,"""c9373e5f772e43d593dd6ad2fa90f6…",107,18144682
18144683,"""c9373e5f772e43d593dd6ad2fa90f6…",43,18144683
…,…,…,…
25043143,"""c5622e0de0594bde95a4dd8c1fcff7…",10,25043143
25043144,"""c5622e0de0594bde95a4dd8c1fcff7…",2,25043144
25043145,"""c5622e0de0594bde95a4dd8c1fcff7…",11,25043145


: 